In [1]:
!pip install --upgrade phospho
! pip install python-dotenv
! pip install openai
! pip install requests

In [2]:
import dotenv
import phospho
import requests
# Load the .env file
dotenv.load_dotenv()

# Check that the required environment variables are set
import os

assert 'PHOSPHO_API_KEY' in os.environ, 'PHOSPHO_API_KEY not set'
assert 'PHOSPHO_PROJECT_ID' in os.environ, 'PHOSPHO_API_SECRET not set'
assert 'OPENAI_API_KEY' in os.environ, 'OPENAI_API_KEY not set'

PHOSPHO_BASE_URL = "http://127.0.0.1:8000"

In [3]:
# Create a dummy agent (random based)
from openai import OpenAI
client = OpenAI() 

def my_answer_generation(user_prompt: str, model_name: str) -> str:
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_prompt}
    ]
    )

    return(completion.choices[0].message.content)

In [4]:
# Try the agent
my_answer_generation("What is the capital of France?", "gpt-4")

'The capital of France is Paris.'

In [5]:
# Init the phospho client
phospho.init()

# Update the function
def my_answer_generation(user_prompt: str, model_name: str) -> str:

    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_prompt}
    ]
    )

    answer = completion.choices[0].message.content

    phospho.log(
        input=user_prompt,
        output=answer,
    )

    return answer

In [8]:
# Create an AB test

# Define the JSON payload for the ABTestCreationRequest
payload = {
    "name": "Test A vs. Test B",
    "description": "A description of the AB test comparing variations A and B.",
    "variations": [
        {"variation_name": "GPT 3.5 turbo", "allocation_rate": 0.5, "features": {"model_name": "gpt-3.5-turbo"}},
        {"variation_name": "GPT 4", "allocation_rate": 0.5, "features": {"model_name": "gpt-4"}}
    ]
}

# Include the necessary headers, such as Authorization if needed
headers = {
    "Authorization": f"Bearer {os.environ['PHOSPHO_API_KEY']}",
    "Content-Type": "application/json"
}

# Send the POST request
url = f"{PHOSPHO_BASE_URL}/v2/projects/{os.environ['PHOSPHO_PROJECT_ID']}/ab-tests"
response = requests.post(url, json=payload, headers=headers)

# Check the response
if response.status_code == 200:
    print("AB test created successfully.")
    print("Response:", response.json())
    ab_test_id = response.json()["id"]
else:
    print("Failed to create AB test. Status code:", response.status_code)
    print("Response:", response.text)

AB test created successfully.
Response: {'id': '9c5c5c4260fc4aca9ee7da349298a9c6', 'project_id': '6d03f091290b4719809c489b78c2344b', 'org_id': '3fe248a3-834c-4c26-8dcc-4e55112f702d', 'name': 'Test A vs. Test B', 'description': 'A description of the AB test comparing variations A and B.', 'created_at': 1707464331, 'terminated_at': None, 'status': 'started', 'variations': [{'variation_name': 'GPT 3.5 turbo', 'allocation_rate': 0.5, 'features': {'model_name': 'gpt-3.5-turbo'}}, {'variation_name': 'GPT 4', 'allocation_rate': 0.5, 'features': {'model_name': 'gpt-4'}}], 'summary': {}}


In [9]:
ab_test_id = "9c5c5c4260fc4aca9ee7da349298a9c6"

In [15]:
# Add the AB testing in our function
import random

# Get the AB testing from the API
# Include the necessary headers, such as Authorization if needed
headers = {
    "Authorization": f"Bearer {os.environ['PHOSPHO_API_KEY']}",
    "Content-Type": "application/json"
}

# Send the POST request
url = f"{PHOSPHO_BASE_URL}/v2/ab-tests/{ab_test_id}"
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    ab_test = response.json()
else:
    raise ValueError("Failed to create AB test. Status code:", response.status_code)

# Update the function
def my_answer_generation(user_prompt: str, model_name: str) -> str:

    # Get the variations from the AB test
    variations = ab_test["variations"]

    # Get a random float between 0 and 1
    random_number = random.random()

    # Select the corresponding variation
    cumulated_rate = 0
    for variation in variations:
        cumulated_rate += variation["allocation_rate"]
        if random_number <= cumulated_rate:
            model_name = variation["features"]["model_name"]
            break
    
    # Optional: log the model name
    print(f"Using model {model_name}")

    completion = client.chat.completions.create(
    model=model_name, # Use the model_name from the AB test
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_prompt}
    ]
    )

    answer = completion.choices[0].message.content

    phospho.log(
        input=user_prompt,
        output=answer,
    )

    return answer
